<a href="https://colab.research.google.com/github/rebeccaxchu/Montgomerty-County-Real-Property-Assessments/blob/main/ETL_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
#import libraries
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# specify the path to your JSON configuration file
config_file_path= '/content/gdrive/MyDrive/Colab Notebooks/mdproperty/credentials.json'

# load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

Connection_String_AZURE_STORAGE = config["ConnectionString"]
CONTAINER_AZURE= 'mdrealproperty'

In [ ]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [ ]:
#initialize the blobserviceclient
blob_service_client = BlobServiceClient.from_connection_string(Connection_String_AZURE_STORAGE)

#get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

In [ ]:
#list all blobs in specified container
blob_list = container_client.list_blobs()

for blob in blob_list:
  print(blob.name)
  blob_client = container_client.get_blob_client(blob=blob.name)
  blob_data = blob_client.download_blob()
  blob_content = blob_data.readall().decode('utf-8')
  df = pd.read_csv(StringIO(blob_content))

property.csv


<ipython-input-14-81df1378b9d7>:9: DtypeWarning: Columns (21,47,89,124,166,173,178,186,199,201,203,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


In [ ]:
# Database connection URL
username = config["username"]
password = config["password"]
host = config["host"]
port = 5432
dbname = 'postgres'

database_url = f'postgresql://{username}:{password}@{host}:{port}/{dbname}?sslmode=require'

engine = create_engine(database_url)

In [ ]:
  # Display the head of the DataFrame
  df.shape

(35228, 209)

In [ ]:
df.dropna()

,land_use_code_mdp_field_lu_desclu_sdat_field_50,new_construction_data_county_assessment_credit_sdat_field_192,parent_account_number_account_number_sdat_field_388,additional_c_a_m_a_data_dwelling_construction_code_mdp_field_strucnst_sdat_field_263,last_activity_date_yyyy_mm_dd_sdat_field_392,base_cycle_data_improvements_value_sdat_field_155,base_cycle_data_preferential_land_value_sdat_field_156,parent_account_number_district_ward_sdat_field_387,special_dates_and_data_inchoate_lien_date_yyyy_mm_dd_sdat_field_209,record_key_owner_occupancy_code_mdp_field_ooi_sdat_field_6,...,section_mdp_field_section_sdat_field_39,premise_address_direction_mdp_field_premsdir_sdat_field_22,special_dates_and_data_non_perc_area_unit_mdp_field_nprcluom_sdat_field_220,base_cycle_data_assessor_code_sdat_field_160,premise_address_number_suffix_sdat_field_21,record_key_subdistrict_sdat_field_4,municipal_tax_class_sdat_field_59,sales_segment_1_grantor_deed_reference_1_clerk_initials_mdp_field_gr1clrk1_sdat_field_81,special_dates_and_data_critical_area_code_mdp_field_crtarcod_sdat_field_223,sales_segment_2_grantor_deed_reference_1_clerk_initials_sdat_field_101


In [ ]:
 # since I have only one csv, I am doing to do the following instructions
md_property_df = df.copy()


In [ ]:
# Get the DataFrame's dtypes, sort by column names
sorted_dtypes = md_property_df.dtypes.sort_index()

# Print each sorted column name with its data type
for column, dtype in sorted_dtypes.items():
    print(f"{column}: {dtype}")


account_id_mdp_field_acctid: int64
ad_valorem_code_sdat_field_62: int64
additional_c_a_m_a_data_building_style_code_and_description_mdp_field_strustyl_descstyl_sdat_field_264: object
additional_c_a_m_a_data_dwelling_construction_code_mdp_field_strucnst_sdat_field_263: object
additional_c_a_m_a_data_dwelling_type_mdp_field_strubldg_sdat_field_265: object
additional_c_a_m_a_data_land_valuation_unit_sdat_field_266: object
additional_c_a_m_a_data_permit_type_mdp_field_permittyp_sdat_field_262: object
assessment_credit_program_current_county_assmt_cr_sdat_field_199: int64
assessment_credit_program_current_credit_status_code_sdat_field_202: object
assessment_credit_program_current_municipal_assmt_cr_sdat_field_201: int64
assessment_credit_program_current_state_assmt_cr_sdat_field_197: int64
assessment_credit_program_previous_county_assmt_cr_sdat_field_198: int64
assessment_credit_program_previous_credit_status_code_sdat_field_203: object
assessment_credit_program_previous_municipal_assmt_cr_

In [ ]:
# first we start with remapping columns for property features
property_mapping = {
    'account_id_mdp_field_acctid': 'property_id',
    'mdp_latitude_mdp_field_digycord_converted_to_wgs84': 'latitude',
    'mdp_longitude_mdp_field_digxcord_converted_to_wgs84': 'longitude',
    'mdp_street_address_city_mdp_field_city': 'city',
    'mdp_street_address_zip_code_mdp_field_zipcode': 'zipcode',
    'c_a_m_a_system_data_dwelling_condition_code_sdat_field_233': 'dwelling_condition',
    'c_a_m_a_system_data_dwelling_grade_code_and_description_mdp_field_strugrad_strudesc_sdat_field_230': 'dwelling_grade_code_and_description',
    'c_a_m_a_system_data_effective_depth_mdp_field_depth_sdat_field_245': 'field_depth',
    'c_a_m_a_system_data_effective_width_mdp_field_width_sdat_field_244': 'field_width',
    'c_a_m_a_system_data_land_area_mdp_field_landarea_sdat_field_242': 'land_area',
    'c_a_m_a_system_data_land_unit_of_measure_mdp_field_luom_sdat_field_243': 'land_unit_of_measure',
    'c_a_m_a_system_data_number_of_dwelling_units_mdp_field_bldg_units_sdat_field_239': 'number_of_dwelling_units',
    'c_a_m_a_system_data_number_of_stories_mdp_field_bldg_story_sdat_field_240': 'number_of_stories',
    'c_a_m_a_system_data_set_no_sdat_field_236': 'set_no',
    'c_a_m_a_system_data_structure_area_sq_ft_mdp_field_sqftstrc_sdat_field_241': 'structure_area_sq_ft',
    'c_a_m_a_system_data_subset_no_sdat_field_237': 'subset_no',
    'c_a_m_a_system_data_year_built_yyyy_mdp_field_yearblt_sdat_field_235': 'year_built',
    'additional_c_a_m_a_data_building_style_code_and_description_mdp_field_strustyl_descstyl_sdat_field_264': 'building_style',
    'additional_c_a_m_a_data_dwelling_construction_code_mdp_field_strucnst_sdat_field_263': 'dwelling_construction',
    'additional_c_a_m_a_data_dwelling_type_mdp_field_strubldg_sdat_field_265': 'dwelling_type',
    'additional_c_a_m_a_data_land_valuation_unit_sdat_field_266': 'valuation_unit',
    'additional_c_a_m_a_data_permit_type_mdp_field_permittyp_sdat_field_262': 'permit_type',
    'land_use_code_mdp_field_lu_desclu_sdat_field_50': 'land_use',
    'property_factors_influence_commer_indust_mdp_field_pfic_sdat_field_69': 'commercial_industrial_influence',
    'property_factors_influence_historical_mdp_field_pfih_sdat_field_70': 'historical_influence',
    'property_factors_location_waterfront_mdp_field_pflw_sdat_field_65': 'waterfront_location',
    'property_factors_street_paved_mdp_field_pfsp_sdat_field_67': 'street_paved',
    'property_factors_street_unpaved_mdp_field_pfsu_sdat_field_68': 'street_unpaved',
    'property_factors_utilities_sewer_mdp_field_pfus_sdat_field_64': 'utilities_sewer',
    'property_factors_utilities_water_mdp_field_pfuw_sdat_field_63': 'utilities_water'
}

In [ ]:
# Select only the columns specified in the dictionary keys from the original DataFrame
filtered_df = md_property_df[list(property_mapping.keys())]

# Rename the columns in this filtered DataFrame
property_df = filtered_df.rename(columns=property_mapping)

# Get the DataFrame's dtypes, sort by column names
sorted_dtypes = property_df.dtypes.sort_index()

# Print each sorted column name with its data type
for column, dtype in sorted_dtypes.items():
    print(f"{column}: {dtype}")


building_style: object
city: object
commercial_industrial_influence: object
dwelling_condition: object
dwelling_construction: object
dwelling_grade_code_and_description: object
dwelling_type: object
field_depth: int64
field_width: int64
historical_influence: object
land_area: float64
land_unit_of_measure: object
land_use: object
latitude: float64
longitude: float64
number_of_dwelling_units: int64
number_of_stories: int64
permit_type: object
property_id: int64
set_no: int64
street_paved: object
street_unpaved: object
structure_area_sq_ft: int64
subset_no: int64
utilities_sewer: object
utilities_water: float64
valuation_unit: object
waterfront_location: object
year_built: int64
zipcode: float64


In [ ]:
property_df

,property_id,latitude,longitude,city,zipcode,dwelling_condition,dwelling_grade_code_and_description,field_depth,field_width,land_area,...,permit_type,land_use,commercial_industrial_influence,historical_influence,waterfront_location,street_paved,street_unpaved,utilities_sewer,utilities_water,commercial_industrial_influence_id
0,160403753156,NaN,NaN,NaN,NaN,Blank (0),NaN,0,0,104855.0,...,NaN,Industrial (I),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),NaN,NaN,0
1,160403872647,NaN,NaN,NaN,NaN,Blank (0),NaN,0,0,13944.0,...,NCP,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),NaN,NaN,0
2,160403339030,39.094875,-77.191403,ROCKVILLE,20850.0,Blank (0),Above Average (5),0,0,1497.0,...,Z,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),Public Sewer (1),NaN,0
3,160500253572,39.078719,-76.958061,SILVER SPRING,20904.0,Blank (0),Average (4),0,0,43680.0,...,NaN,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),Public Sewer (1),NaN,0
4,160403366955,39.116729,-77.173089,ROCKVILLE,20850.0,Blank (0),Above Average (5),0,0,2288.0,...,Z,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),Public Sewer (1),NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35223,160203692616,39.230294,-77.248340,CLARKSBURG,20871.0,Blank (0),NaN,0,0,0.0,...,NaN,Residential Condominium (U),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),NaN,NaN,0
35224,160203692970,39.225250,-77.237117,CLARKSBURG,20871.0,Blank (0),NaN,0,0,5153.0,...,NaN,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),NaN,NaN,0
35225,160203692981,39.225359,-77.237796,CLARKSBURG,20871.0,Blank (0),Above Average (5),0,0,8957.0,...,RAP,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),NaN,NaN,0
35226,160203692992,39.225301,-77.237565,CLARKSBURG,20871.0,Blank (0),Above Average (5),0,0,8704.0,...,NaN,Residential (R),None (0),No Historical Influence (0),None (0),No Paved Street (0),No Unpaved Street (0),NaN,NaN,0


In [ ]:
# Columns to be factorized
columns_to_factorize = [
    'commercial_industrial_influence',
    'dwelling_condition',
    'dwelling_construction',
    'dwelling_grade_code_and_description',
    'dwelling_type',
    'permit_type',
    'street_paved',
    'utilities_sewer',
    'valuation_unit',
    'waterfront_location'
]

for column in columns_to_factorize:
    # Factorizing the column
    ids, uniques = pd.factorize(property_df[column])

    # Adding the ID column to the original DataFrame
    property_df[column + '_id'] = ids

    # Creating a dimension DataFrame for the current column
    dim_name = 'dim_' + column
    globals()[dim_name] = pd.DataFrame({
        column: uniques,
        column + '_id': range(len(uniques))
    })

    # Exporting the dimension DataFrame to SQL
    globals()[dim_name].to_sql(dim_name, engine, if_exists='replace', index=False)

    # Printing the name and columns of the DataFrame
    print(f"DataFrame Name: {dim_name}")
    print("Columns:", globals()[dim_name].columns.tolist())
    print()

print("All dimension tables have been exported to the SQL database.")


DataFrame Name: dim_commercial_industrial_influence
Columns: ['commercial_industrial_influence', 'commercial_industrial_influence_id']

DataFrame Name: dim_dwelling_condition
Columns: ['dwelling_condition', 'dwelling_condition_id']

DataFrame Name: dim_dwelling_construction
Columns: ['dwelling_construction', 'dwelling_construction_id']

DataFrame Name: dim_dwelling_grade_code_and_description
Columns: ['dwelling_grade_code_and_description', 'dwelling_grade_code_and_description_id']

DataFrame Name: dim_dwelling_type
Columns: ['dwelling_type', 'dwelling_type_id']

DataFrame Name: dim_permit_type
Columns: ['permit_type', 'permit_type_id']

DataFrame Name: dim_street_paved
Columns: ['street_paved', 'street_paved_id']

DataFrame Name: dim_utilities_sewer
Columns: ['utilities_sewer', 'utilities_sewer_id']

DataFrame Name: dim_valuation_unit
Columns: ['valuation_unit', 'valuation_unit_id']

DataFrame Name: dim_waterfront_location
Columns: ['waterfront_location', 'waterfront_location_id']

All

In [ ]:
# Now we proceed to location and start with zipcode
# Replace NaN with a placeholder, then convert to int
property_df['zipcode'] = property_df['zipcode'].fillna(0).astype(int)

In [ ]:
location_data = property_df[['zipcode', 'city']]
dim_location = location_data.drop_duplicates(subset=['zipcode'])

# ensure there are no missing values in the primary key column
dim_location = dim_location.dropna(subset=['zipcode'])

# To set 'zipcode' as the index of the DataFrame, which simulates a primary key in a database:
dim_location.set_index('zipcode', inplace=True)


property_df['zipcode']=

                  city
zipcode               
0                  NaN
20850        ROCKVILLE
20904    SILVER SPRING
20878     GAITHERSBURG
20817         BETHESDA
...                ...
20770       GERMANTOWN
20864       GERMANTOWN
20787       CLARKSBURG
20891            BOYDS
22102       CLARKSBURG

[62 rows x 1 columns]


In [ ]:
dim_location.to_sql('dim_location', engine, if_exists='append', index=False)

62

In [ ]:
# move on to fact table
fact_mapping = {
    'account_id_mdp_field_acctid': 'fact_id',
    'base_cycle_data_date_assessed_yyyy_mm_sdat_field_158': 'base_cycle_data_date',
    'prior_assessment_year_total_assessment_sdat_field_161': 'prior_assessment_total',
    'current_cycle_data_date_assessed_yyyy_mm_mdp_field_lastassd_sdat_field_169': 'current_cycle_data_date',
    'current_assessment_year_total_assessment_sdat_field_172': 'current_assessment_total'
}


In [ ]:
# Select only the columns specified in the dictionary keys from the original DataFrame
filtered_df = md_property_df[list(fact_mapping.keys())]

# Rename the columns in this filtered DataFrame
fact_df = filtered_df.rename(columns=fact_mapping)

# Display the first few rows of the new DataFrame to confirm the changes
print(fact_df.head())


        fact_id  base_cycle_data_date  prior_assessment_total  \
0  160403753156               2021.01                 9375000   
1  160403872647               2022.01                 1254900   
2  160403339030               2022.01                  585867   
3  160500253572               2021.01                  506700   
4  160403366955               2022.01                  614733   

   current_cycle_data_date  current_assessment_total  
0                  2024.01                   9375000  
1                  2022.01                   1254900  
2                  2022.01                    595300  
3                  2024.01                    548233  
4                  2022.01                    628500  


In [ ]:
# add columns of value change in dollar and percentage

# Calculate 'value_change' by subtracting 'prior_assessment_total' from 'current_assessment_total'
fact_df['value_change'] = fact_df['current_assessment_total'] - fact_df['prior_assessment_total']

# Calculate 'percentage_change'
# Check for division by zero by replacing zeros in the denominator with NaN, which will result in NaN for that calculation
fact_df['percentage_change'] = (fact_df['value_change'] / fact_df['prior_assessment_total'].replace(0, np.nan)) * 100

In [ ]:
columns_to_add = [
    'field_depth', 'field_width', 'land_area', 'latitude', 'longitude',
    'number_of_dwelling_units', 'number_of_stories', 'property_id', 'set_no',
    'structure_area_sq_ft', 'subset_no', 'utilities_water', 'year_built', 'zipcode'
]

In [ ]:
for item in columns_to_add:
  fact_df[item]=property_df[item]

In [ ]:
fact_df['commercial_industrial_influence_id']=dim_commercial_industrial_influence['commercial_industrial_influence_id']
fact_df['dwelling_condition_id'] = dim_dwelling_condition['dwelling_condition_id']
fact_df['dwelling_construction_id'] = dim_dwelling_construction['dwelling_construction_id']
fact_df['dwelling_grade_code_and_description_id'] = dim_dwelling_grade_code_and_description['dwelling_grade_code_and_description_id']
fact_df['dwelling_type_id'] = dim_dwelling_type['dwelling_type_id']
fact_df['permit_type_id'] = dim_permit_type['permit_type_id']
fact_df['street_paved_id'] = dim_street_paved['street_paved_id']
fact_df['utilities_sewer_id'] = dim_utilities_sewer['utilities_sewer_id']
fact_df['valuation_unit_id'] = dim_valuation_unit['valuation_unit_id']
fact_df['waterfront_location_id'] = dim_waterfront_location['waterfront_location_id']

In [79]:
# Get the DataFrame's dtypes, sort by column names
sorted_dtypes = fact_df.dtypes.sort_index()

# Print each sorted column name with its data type
for column, dtype in sorted_dtypes.items():
    print(f"{column}: {dtype}")


base_cycle_data_date: float64
commercial_industrial_influence_id: float64
current_assessment_total: int64
current_cycle_data_date: float64
dwelling_condition_id: float64
dwelling_construction_id: float64
dwelling_grade_code_and_description_id: float64
dwelling_type_id: float64
fact_id: int64
field_depth: int64
field_width: int64
land_area: float64
latitude: float64
longitude: float64
number_of_dwelling_units: int64
number_of_stories: int64
percentage_change: float64
permit_type_id: float64
prior_assessment_total: int64
property_id: int64
set_no: int64
street_paved_id: float64
structure_area_sq_ft: int64
subset_no: int64
utilities_sewer_id: float64
utilities_water: float64
valuation_unit_id: float64
value_change: int64
waterfront_location_id: float64
year_built: int64
zipcode: int64


In [ ]:
print(fact_df.head())

        fact_id  base_cycle_data_date  prior_assessment_total  \
0  160403753156               2021.01                 9375000   
1  160403872647               2022.01                 1254900   
2  160403339030               2022.01                  585867   
3  160500253572               2021.01                  506700   
4  160403366955               2022.01                  614733   

   current_cycle_data_date  current_assessment_total  value_change  \
0                  2024.01                   9375000             0   
1                  2022.01                   1254900             0   
2                  2022.01                    595300          9433   
3                  2024.01                    548233         41533   
4                  2022.01                    628500         13767   

   percentage_change  field_depth  field_width  land_area  ...  \
0           0.000000            0            0   104855.0  ...   
1           0.000000            0            0    13944.

In [ ]:
fact_df.to_sql('fact', engine, if_exists='replace', index=False)

228